<p><i>Preleva questo notebook da GitHub oppure clic sul pulsante sopra</i></p> <a href="https://github.com/lrnzr/didatticaRt/blob/main/indiceRiproduzioneRt.ipynb"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download notebook da GitHub"></a><br>
<p><i>Per una consultazione interattiva</i></p> <a href="https://mybinder.org/v2/gh/lrnzr/didatticaRt/master"><img align="left" src="https://static.mybinder.org/badge_logo.svg" alt="Vai al sito MyBinder.org" title="Vai al sito MyBinder.org"></a></p>

# Calcola l'indice di riproduzione <em>R<sub>t</sub></em> e ne rappresenta l'andamento

*Sulla base del tasso di crescita &lambda; (notebook `tassoCrescitaLambda.ipynb`) e su una stima del tempo di generazione si calcola l'indice <em>R</em><sub>t</sub>. Data l'incertezza del tempo di generazione, si individuano le fasce di confidenza al 68%.*

Carica i moduli necessari

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import datetime

La funzione `partiziona()` partiziona `xDati` in un array di array. Ciascun elemento ha lunghezza `intervalloGiorni` e il loro numero è pari a `n-intervalloGiorni+1`. Segue la funzione per il calcolo di <em>R<sub>t</sub></em>.

In [ ]:
def partiziona(xdati, intervalloGiorni):
    numElem = len(xdati)
    elementi = []
    for i in range(intervalloGiorni, numElem + 1):
        elementi = np.append(elementi, xdati[i-intervalloGiorni:i])
    return np.reshape(elementi, (numElem - intervalloGiorni + 1, intervalloGiorni))

def rT(g, l):
    return np.exp(g*l)

#################################################################################################################

Informazioni da inserire in input per iniziare l'elaborazione dei dati nazionali o regionali memorizzati nei file corrispondenti alla data scelta. Viene inoltre richiesta l'unità temporale da porre in ascissa e se si desiderano visualizzare le fasce di confidenza.

In [ ]:
print()
print("Si intendono elaborare i nuovi positivi nazionali (i) o quelli di una regione (r)? ")
scelta = input(" :> ")
print("Inserire la data nel formato (YYYYMMDD) ")
dataISO = input(" :> ")
print("Si vuole l'asse temporale espresso in mesi (m) o in giorni trascorsi (g) dall'inizio pandemia? ")
unitaAssex = input(" :> ")
print("Si vogliono visualizzare le fasce di errore (s/n)? ")
fasce = input(" :> ")

Vengono composti i nomi dei file e quindi si carica nell'array `nuoviPositivi` i dati rilevati

In [ ]:
if scelta == "i":
    regioneScelta = "ITALIA"
    nomeFile = 'datiNazionali' + dataISO
    # lettura del file csv: la colonna 8 corrisponde al campo nuovi_positivi
    nuoviPositivi = np.genfromtxt(nomeFile + '.csv', delimiter=',', skip_header=1, usecols=8, dtype=int)
else:
    print("Inserire la regione ")
    regioneScelta = input(" :> ")
    regioneScelta = regioneScelta.lower().capitalize()
    nomeFile = 'datiRegionali' + dataISO
    # lettura del file json e riportato l'array ad un array di Numpy
    with open(nomeFile + '.json') as f:
        datiGrezzi = json.load(f)
    nuoviPositivi = []
    for record in datiGrezzi:
        if record['denominazione_regione'] == regioneScelta:
            nuoviPositivi.append(record['nuovi_positivi'])
    nuoviPositivi = np.array(nuoviPositivi)

#################################################################################################################

Dovendo calcolare il logaritmo dei nuoviPositivi e nel caso il rispettivo valore giornaliero sia nullo, lo si pone pari ad 1

In [ ]:
# nel caso compaia il valore 0 o negativo lo si sostituisce con 1 in quanto il logaritmo seguente darebbe errore
nuoviPositivi[nuoviPositivi == 0] = 1
logPositivi = np.log(nuoviPositivi)
numGiorni = len(nuoviPositivi)
intervalloGiorni = 14
# i giorni da 0 a 13 nel quale viene eseguito il fit
giorniDelFit = np.arange(0, intervalloGiorni, 1)

Si applica la regressione lineare in un intervallo temporale di 14 giorni determinando, della retta ottimale, la sola pendenza in quanto associata, nel modello teorico, al parametro &lambda; o tasso di crescita.

In [ ]:
valoriLambdaLin = []
for i in range(intervalloGiorni, numGiorni+1):
    # fit lineare
    esito = linregress(giorniDelFit, logPositivi[i-intervalloGiorni:i])
    valoriLambdaLin = np.append(valoriLambdaLin, esito.slope)

Assegnato al tempo di generazione il valore ripreso dalle pubblicazion1 <a href="#ref01">[1]</a> e <a href="#ref02">[2]</a>, si calcola il valore giornaliero di <em>R<sub>t</sub></em> e quindi si esegue una media mobile su 14 giorni.

In [ ]:
# valori ricavati dalla pubblicazione citata
generationTime = 5.8
valoriMediDayRtLin = rT(generationTime, valoriLambdaLin)
media14Giorni = np.mean(partiziona(valoriMediDayRtLin, intervalloGiorni), axis=1)

Assegnata l'incertezza $\Delta g=1.88$ giorni al tempo di generazione, si ricalcolano i valori di <em>R<sub>t</sub></em> a partire dalle nuove stime $$g-\Delta g=5.88-1.88\qquad e\qquad g+\Delta g=5.88+1.88.$$

In [ ]:
if fasce == 's':
    generationTimeError = 1.88
    valoriMediDayRtLinSup = rT(generationTime+generationTimeError, valoriLambdaLin)
    valoriMediDayRtLinInf = rT(generationTime-generationTimeError, valoriLambdaLin)
    media14GiorniSup = np.mean(partiziona(valoriMediDayRtLinSup, intervalloGiorni), axis=1)
    media14GiorniInf = np.mean(partiziona(valoriMediDayRtLinInf, intervalloGiorni), axis=1)

Predisposizioni per la parte grafica

In [ ]:
rtMassimo = 3
codificaItaliana = dataISO[6:] + '-' + dataISO[4:6] + '-' + dataISO[:4]
xValoriMediGiornalieri = np.arange(intervalloGiorni-1, numGiorni)
xMediaMobile = np.arange(intervalloGiorni+5, numGiorni-intervalloGiorni/2)

Inizia la parte grafica comune e quindi quella dipendente dalle scelte iniziali. Le variabili temporali `date1`, `date2` come le precedenti `xValoriMediGiornalieri` e `xMediaMobile` controllano la traslazione temporale dei valori in ordinata e sono scelte in modo da ottenere un accordo visivo soddisfacente con i dati giornalieri e in coerenza con il sito [CovidStat](https://covid19.infn.it/).

In [ ]:
# parte grafica comune
plt.rcParams['figure.figsize'] = [12, 6]
fig, ax = plt.subplots()
plt.ylim([0.4, rtMassimo])
y_major_ticks = np.arange(0.4, rtMassimo, 0.1)
ax.set_yticks(y_major_ticks)
ax.set_ylabel('indice di riproduzione Rt')
ax.grid(which='both', color='.85', linestyle='-', linewidth=1)
# parte grafica variabile
if fasce == "s":  # assieme all'Rt si visualizzano le fasce di confidenza
    ax.set_title(regioneScelta + ": Indice di riproduzione Rt e intervalli di confidenza 68%")
    if unitaAssex == "m":
        ax.set_xlabel('date dal 24 febbraio 2020')
        date2 = [datetime.date(2020, 3, 8) + datetime.timedelta(days=int(intervalloGiorni/2)) + datetime.timedelta(days=i)
                 for i in range(numGiorni-intervalloGiorni-12)]
        ax.fill_between(date2, media14Giorni, media14GiorniSup, interpolate=True, facecolor='papayawhip')
        ax.fill_between(date2, media14Giorni, media14GiorniInf, interpolate=True, facecolor='papayawhip')
        ax.plot(date2, np.full(xMediaMobile.shape, 1), color='b')
        ax.plot(date2, media14GiorniSup, color='lightskyblue', linewidth=2, label='media mobile Rt ($g+\Delta g$)')
        ax.plot(date2, media14GiorniInf, color='greenyellow', linewidth=2, label='media mobile Rt ($g-\Delta g$)')
        ax.plot(date2, media14Giorni, color='r', linewidth=2, label='media mobile Rt su 14 giorni', zorder=4)
        date1 = [datetime.date(2020, 3, 1) + datetime.timedelta(days=int(intervalloGiorni/2)) + datetime.timedelta(days=i)
                 for i in range(numGiorni-intervalloGiorni+1)]
        ax.scatter(date1, valoriMediDayRtLin, s=4, label='Rt giornaliero', zorder=3)
        ax.text(datetime.date(2020, 11, 1), 0.8*rtMassimo, 'aggiornato il\n' + codificaItaliana)
    else:
        ax.set_xlabel('giorni dal 24 febbraio 2020')
        ax.fill_between(xMediaMobile, media14Giorni, media14GiorniSup, interpolate=True, facecolor='papayawhip')
        ax.fill_between(xMediaMobile, media14Giorni, media14GiorniInf, interpolate=True, facecolor='papayawhip')
        ax.plot(xMediaMobile, np.full(xMediaMobile.shape, 1), color='b')
        ax.plot(xMediaMobile, media14GiorniSup, color='lightskyblue', linewidth=2, label='media mobile Rt ($g+\Delta g$)')
        ax.plot(xMediaMobile, media14GiorniInf, color='greenyellow', linewidth=2, label='media mobile Rt ($g-\Delta g$)')
        ax.plot(xMediaMobile, media14Giorni, color='r', linewidth=2, label='media mobile Rt su 14 giorni', zorder=4)
        ax.scatter(xValoriMediGiornalieri, valoriMediDayRtLin, s=4, label='Rt giornaliero', zorder=3)
        ax.text(140, 0.8*rtMassimo, 'aggiornato il\n' + codificaItaliana)
        x_major_ticks = np.arange(0, numGiorni+20, 20)
        ax.set_xticks(x_major_ticks)
else:  # visualizza solo l'andamento di Rt
    ax.set_title(regioneScelta + ": Indice di riproduzione Rt")
    if unitaAssex == "m":
        ax.set_xlabel('date dal 24 febbraio 2020')
        date2 = [datetime.date(2020, 3, 8) + datetime.timedelta(days=int(intervalloGiorni/2)) + datetime.timedelta(days=i)
                 for i in range(numGiorni-intervalloGiorni-12)]
        ax.plot(date2, np.full(xMediaMobile.shape, 1), color='b')
        ax.plot(date2, media14Giorni, color='r', linewidth=2, label='media mobile Rt su 14 giorni', zorder=4)
        date1 = [datetime.date(2020, 3, 1) + datetime.timedelta(days=int(intervalloGiorni/2)) + datetime.timedelta(days=i)
                 for i in range(numGiorni-intervalloGiorni+1)]
        ax.scatter(date1, valoriMediDayRtLin, s=4, label='Rt giornaliero', zorder=3)
        ax.text(datetime.date(2020, 11, 1), 0.8*rtMassimo, 'aggiornato il\n' + codificaItaliana)
    else:
        ax.set_xlabel('giorni dal 24 febbraio 2020')
        ax.plot(xMediaMobile, np.full(xMediaMobile.shape, 1), color='b')
        ax.plot(xMediaMobile, media14Giorni, color='r', linewidth=2, label='media mobile Rt su 14 giorni', zorder=4)
        ax.scatter(xValoriMediGiornalieri, valoriMediDayRtLin, s=4, label='Rt giornaliero', zorder=3)
        ax.text(140, 0.8*rtMassimo, 'aggiornato il\n' + codificaItaliana)
        x_major_ticks = np.arange(0, numGiorni+20, 20)
        ax.set_xticks(x_major_ticks)

plt.legend()
plt.show()
#################################################################################################################

## Referenze

<p id="ref01">
      [1] G. Bonifazi, L. Lista et al., <a href="https://doi.org/10.1140/epjp/s13360-021-01339-6">A simplified estimate of the Effective Reproduction <em>R</em><sub>t</sub> Number using its relation with the doubling time and application to Italian COVID-19 data</a>, The European Physical Journal Plus (2021)
    </p>

<p id="ref02">
      [2] D. Cereda et al., <a href="https://arxiv.org/abs/2003.09320">The early phase of the COVID-19 outbreak in Lombardy, Italy</a>, arXiv:2003.09320 (2020)
</p>